# 패키지 다운로드

In [1]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import warnings

from statsmodels.stats.outliers_influence import variance_inflation_factor

np.set_printoptions(suppress=True)


mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

warnings.filterwarnings("ignore")

# 데이터 셋 확인

In [ ]:
# '선박번호', '날짜_시간', '날짜', '시간', '위도', '경도', '대지속력', '실침로', '선수미선','점사이_거리', '시간_차이', '정박지', '평균선회반경(m)', '유속', '유향', '우행', '풍향', '풍속', '최대파고', '유의파고', '평균파고', 'answer', '유행'

In [24]:
# 데이터 불러오기
busan = pd.read_csv("./data/busan_train_full3.csv", encoding="cp949")

In [14]:
busan[['선박번호', '날짜_시간','대지속력', '실침로', '선수미선', '점사이_거리', '첫점과의거리', '시간_차이', '정박지', '평균선회반경(m)', '유속', '유행', '풍향', '풍속', '최대파고', '유의파고', '평균파고', 'answer']]

,선박번호,날짜_시간,대지속력,실침로,선수미선,점사이_거리,첫점과의거리,시간_차이,정박지,평균선회반경(m),유속,유향,유행,풍향,풍속,최대파고,유의파고,평균파고,answer
0,1,2021-01-14 22:47,0.5,123.8,NaN,0.000000,0.000000,NaN,3,322.0,21.3,0.0,98.0,358.0,1.2,NaN,NaN,NaN,0
1,1,2021-01-14 22:50,0.2,131.7,NaN,0.040560,40.559515,0 days 00:03:00,3,322.0,21.1,0.0,95.0,185.0,0.2,NaN,NaN,NaN,0
2,1,2021-01-14 22:56,0.2,255.9,NaN,0.003781,43.869680,0 days 00:06:00,3,322.0,19.6,0.0,92.0,284.0,0.1,NaN,NaN,NaN,0
3,1,2021-01-14 23:02,0.1,280.3,NaN,0.018167,40.146094,0 days 00:06:00,3,322.0,23.4,0.0,100.0,311.0,0.9,NaN,NaN,NaN,0
4,1,2021-01-14 23:11,0.1,157.6,NaN,0.033778,72.408202,0 days 00:09:00,3,322.0,20.7,0.0,90.0,264.0,1.7,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251015,247,2022-04-10 17:36,0.1,125.3,222.0,0.002257,8.963695,0 days 00:03:00,3,322.0,19.1,8.0,71.0,198.0,10.1,NaN,NaN,NaN,0
251016,247,2022-04-10 17:39,0.1,149.6,223.0,0.003960,5.169480,0 days 00:03:00,3,322.0,16.9,8.0,61.0,196.0,10.6,NaN,NaN,NaN,0
251017,247,2022-04-10 17:45,0.1,152.4,223.0,0.004105,2.751241,0 days 00:06:00,3,322.0,18.2,8.0,73.0,195.0,9.2,NaN,NaN,NaN,0
251018,247,2022-04-10 17:48,0.1,172.6,223.0,0.005634,6.369583,0 days 00:03:00,3,322.0,15.9,8.0,73.0,196.0,9.4,NaN,NaN,NaN,0


## 유향 전처리 다시하기

In [ ]:
# '선박번호', '날짜_시간', '날짜', '시간', '위도', '경도', '대지속력', '실침로', '선수미선', '점사이_거리', '시간_차이', '정박지', '평균선회반경(m)', '유속', '유향', '우행', '풍향', '풍속', '최대파고', '유의파고', '평균파고', 'answer', '유행', 'Combined'

In [25]:
# 결측치 없는 데이터 정제 / datetime 변환
busan_test = busan
busan_test = busan_test[['선박번호', '날짜_시간','대지속력', '실침로', '선수미선', '점사이_거리', '첫점과의거리', '시간_차이', '정박지', '평균선회반경(m)', '유속', '유행', '풍향', '풍속', 'answer']]
busan_test = busan_test.dropna().reset_index(drop= True)
busan_test

busan_test_full["날짜_시간"] = pd.to_datetime(busan_test_full["날짜_시간"])

,선박번호,날짜_시간,대지속력,실침로,선수미선,점사이_거리,첫점과의거리,시간_차이,정박지,평균선회반경(m),유속,유행,풍향,풍속,answer
0,2,2021-01-16 23:49,0.3,193.9,323.0,0.038190,38.189938,0 days 00:03:00,5,632.5,14.6,99.0,311.0,4.7,0
1,2,2021-01-16 23:55,0.6,44.8,333.0,0.027636,10.695936,0 days 00:06:00,5,632.5,13.5,79.0,311.0,8.0,0
2,2,2021-01-16 23:58,0.7,77.8,325.0,0.034370,28.045527,0 days 00:03:00,5,632.5,17.3,90.0,296.0,7.2,0
3,2,2021-01-17 0:07,0.3,213.2,296.0,0.018121,26.477634,0 days 00:09:00,5,632.5,15.9,88.0,309.0,8.2,0
4,2,2021-01-17 0:10,0.5,201.6,308.0,0.044486,21.316213,0 days 00:03:00,5,632.5,15.9,98.0,307.0,10.8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222861,247,2022-04-10 17:36,0.1,125.3,222.0,0.002257,8.963695,0 days 00:03:00,3,322.0,19.1,71.0,198.0,10.1,0
222862,247,2022-04-10 17:39,0.1,149.6,223.0,0.003960,5.169480,0 days 00:03:00,3,322.0,16.9,61.0,196.0,10.6,0
222863,247,2022-04-10 17:45,0.1,152.4,223.0,0.004105,2.751241,0 days 00:06:00,3,322.0,18.2,73.0,195.0,9.2,0
222864,247,2022-04-10 17:48,0.1,172.6,223.0,0.005634,6.369583,0 days 00:03:00,3,322.0,15.9,73.0,196.0,9.4,0


In [9]:
# 보간법
df = pd.DataFrame()
vessle_num = busan_test_full["선박번호"].unique()

for i in vessle_num:
    temp = busan_test_full[busan_test_full["선박번호"] == i]

    temp = temp.set_index('날짜_시간')
    temp = temp.resample('1T').mean().interpolate()
    df = pd.concat([df, temp], axis = 0)
    
# 보간법으로 늘어난 answer의 값을 0, 1로 조정
df.loc[df['answer'] != 1, 'answer'] = 0
df= df.reset_index()
df

,날짜_시간,선박번호,정박지,유속,유행,풍향,풍속,answer
0,2021-01-14 22:47:00,1.0,3.0,21.300000,98.000000,358.000000,1.200000,0.0
1,2021-01-14 22:48:00,1.0,3.0,21.233333,97.000000,300.333333,0.866667,0.0
2,2021-01-14 22:49:00,1.0,3.0,21.166667,96.000000,242.666667,0.533333,0.0
3,2021-01-14 22:50:00,1.0,3.0,21.100000,95.000000,185.000000,0.200000,0.0
4,2021-01-14 22:51:00,1.0,3.0,20.850000,94.500000,201.500000,0.183333,0.0
...,...,...,...,...,...,...,...,...
338735,2022-04-10 17:47:00,247.0,3.0,16.666667,73.000000,195.666667,9.333333,0.0
338736,2022-04-10 17:48:00,247.0,3.0,15.900000,73.000000,196.000000,9.400000,0.0
338737,2022-04-10 17:49:00,247.0,3.0,17.100000,74.666667,197.000000,9.033333,0.0
338738,2022-04-10 17:50:00,247.0,3.0,18.300000,76.333333,198.000000,8.666667,0.0


In [67]:
# 상관관계 돌려보기 (연속형 변수끼리만)
busan_1 = busan_test_full[['대지속력', '실침로', '선수미선', '유속', 'Combined', '풍향', '풍속']]
busan_corr = busan_1.corr()
busan_corr

,대지속력,실침로,선수미선,유속,Combined,풍향,풍속
대지속력,1.000000,-0.004360,-0.030474,-0.001514,-0.007437,-0.015800,0.274774
실침로,-0.004360,1.000000,0.014183,-0.004762,-0.023509,-0.076499,0.023524
선수미선,-0.030474,0.014183,1.000000,0.076865,-0.167962,0.436558,0.080317
유속,-0.001514,-0.004762,0.076865,1.000000,-0.202303,-0.048594,-0.033474
Combined,-0.007437,-0.023509,-0.167962,-0.202303,1.000000,-0.025460,0.001356
풍향,-0.015800,-0.076499,0.436558,-0.048594,-0.025460,1.000000,-0.029534
풍속,0.274774,0.023524,0.080317,-0.033474,0.001356,-0.029534,1.000000


In [32]:
## 상관관계 돌려보기 (명목형 변수 더미처리)
dummy = pd.get_dummies(busan_test_full['정박지'])

dummy_busan = pd.concat([dummy, busan_test_full[['유속', '유행', '풍향', '풍속']]], axis = 1)

busan_corr = dummy_busan.corr()
busan_corr.to_csv("부산_상관계수.csv", encoding="cp949")

In [68]:
# VIF 계산
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    busan_1.values, i) for i in range(busan_1.shape[1])]
vif["features"] = busan_2.columns
vif

,VIF Factor,features
0,1.404051,대지속력
1,3.297483,실침로
2,3.891299,선수미선
3,2.725069,유속
4,3.015036,유향
5,3.535820,풍향
6,3.612072,풍속


In [34]:
# VIF 계산 - 더미데이터 포함
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    dummy_busan.values, i) for i in range(dummy_busan.shape[1])]
vif["features"] = dummy_busan.columns
vif.to_csv("부산_vif.csv", encoding="cp949")

### 보간 & 더미 데이터

In [7]:
dummy = pd.get_dummies(df['정박지'], drop_first = True)

dummy_busan = pd.concat([dummy, df[['선박번호', '유속', 'Combined', '풍향', '풍속', 'answer']]], axis = 1)

### 학습데이터 준비

In [8]:
busan_df = dummy_busan[['선박번호', "유속","Combined","풍향","풍속",2,3,4,5,"answer"]]
busan_df

,선박번호,유속,Combined,풍향,풍속,2.0,3.0,4.0,5.0,answer
0,1.0,21.300000,98.000000,358.000000,1.200000,0,1,0,0,0.0
1,1.0,21.233333,97.000000,300.333333,0.866667,0,1,0,0,0.0
2,1.0,21.166667,96.000000,242.666667,0.533333,0,1,0,0,0.0
3,1.0,21.100000,95.000000,185.000000,0.200000,0,1,0,0,0.0
4,1.0,20.850000,94.500000,201.500000,0.183333,0,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...
338735,247.0,16.666667,73.000000,195.666667,9.333333,0,1,0,0,0.0
338736,247.0,15.900000,73.000000,196.000000,9.400000,0,1,0,0,0.0
338737,247.0,17.100000,74.666667,197.000000,9.033333,0,1,0,0,0.0
338738,247.0,18.300000,76.333333,198.000000,8.666667,0,1,0,0,0.0


## 유향 전처리 안한 데이터
- 위에서 유향 데이터 전처리 이후 진행 안함

In [118]:
# 결측치 없는 데이터 정제
busan_full = busan.dropna().reset_index(drop = True)
busan_full

,선박번호,날짜_시간,날짜,시간,위도,경도,대지속력,실침로,선수미선,점사이_거리,...,유향,우행,풍향,풍속,최대파고,유의파고,평균파고,answer,유행,Combined


### 명목 - 연속 상관관계 분석을 위한 더미데이터 생성

In [38]:
dummy_busan = pd.get_dummies(busan_full['정박지'], drop_first=True)
pd.concat([dummy_busan, busan_full[['대지속력', '실침로', '선수미선', '유속', '유향', '풍향', '풍속']]], axis = 1)

,2,3,4,5,대지속력,실침로,선수미선,유속,유향,풍향,풍속
0,0,0,0,1,0.6,194.8,309.0,13.9,0.0,321.0,4.9
1,0,0,0,1,0.3,193.9,323.0,14.6,0.0,311.0,4.7
2,0,0,0,1,0.6,44.8,333.0,13.5,8.0,311.0,8.0
3,0,0,0,1,0.7,77.8,325.0,17.3,0.0,296.0,7.2
4,0,0,0,1,0.3,213.2,296.0,15.9,0.0,309.0,8.2
...,...,...,...,...,...,...,...,...,...,...,...
223501,0,1,0,0,0.1,125.3,222.0,19.1,8.0,198.0,10.1
223502,0,1,0,0,0.1,149.6,223.0,16.9,8.0,196.0,10.6
223503,0,1,0,0,0.1,152.4,223.0,18.2,8.0,195.0,9.2
223504,0,1,0,0,0.1,172.6,223.0,15.9,8.0,196.0,9.4


In [34]:
# 상관관계 돌려보기
busan_2 = busan_full[['대지속력', '실침로', '선수미선', '유속', '유향', '풍향', '풍속']]
busan_corr = busan_2.corr()
busan_corr

,대지속력,실침로,선수미선,유속,유향,풍향,풍속
대지속력,1.000000,-0.004360,-0.030474,-0.001514,0.016984,-0.015800,0.274774
실침로,-0.004360,1.000000,0.014183,-0.004762,-0.004089,-0.076499,0.023524
선수미선,-0.030474,0.014183,1.000000,0.076865,-0.099973,0.436558,0.080317
유속,-0.001514,-0.004762,0.076865,1.000000,-0.463470,-0.048594,-0.033474
유향,0.016984,-0.004089,-0.099973,-0.463470,1.000000,-0.033367,0.068429
풍향,-0.015800,-0.076499,0.436558,-0.048594,-0.033367,1.000000,-0.029534
풍속,0.274774,0.023524,0.080317,-0.033474,0.068429,-0.029534,1.000000


In [33]:
# VIF 계산
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    busan_2.values, i) for i in range(busan_2.shape[1])]
vif["features"] = busan_2.columns
vif

,VIF Factor,features
0,1.404383,대지속력
1,3.242192,실침로
2,3.878666,선수미선
3,2.875087,유속
4,1.903059,유향
5,3.413757,풍향
6,3.639118,풍속


# 데이터 표준화
- 전체 데이터를 표준화

In [9]:
학습길이 = 60
예측기간 = 30
train_test_split = 200000

vessle_num = busan_df["선박번호"].unique()

In [10]:
# 학습데이터
data = []

for j in vessle_num:
    for i in range(0, len(busan_df[busan_df["선박번호"] == j]) 
                   - 학습길이 - 예측기간):
        temp = busan_df[busan_df["선박번호"] == j]

        dataset = temp.values[i:i+학습길이, 1:5]
        data_mean = dataset.mean(axis=0)
        data_std = dataset.std(axis=0)
        dataset = (dataset-data_mean)/data_std

        b = temp.values[i:i+학습길이, 5:]

        dataset = np.column_stack((dataset, b))

        data.append(dataset)
    
data = np.array(data)

print(data.shape)

(318850, 60, 9)


In [11]:
# 정답 데이터
label = []

for j in vessle_num:
    for i in range(0, len(busan_df[busan_df["선박번호"] == j]) 
                   - 학습길이 - 예측기간):
        temp = busan_df[busan_df["선박번호"] == j]

        dataset = temp.values[학습길이 + 예측기간, -1]
        
        label.append(dataset)
        
label = np.array(label)
print(label.shape)

(318850,)


In [26]:
# 학습 - 평가 데이터 분리
data_split = np.split(data, [train_test_split], axis=0)
label_split = np.split(label, [train_test_split], axis=0)

data_train = data_split[0]
data_test = data_split[1]

label_train = label_split[0]
label_test = label_split[1]

# 결과 확인
print(data_train.shape)  # (200000, 60, 9)
print(data_test.shape)  # (118850, 60, 9)

print(label_train.shape)  # (200000,)
print(label_test.shape)  # (118850,)

(200000, 60, 9)
(118850, 60, 9)
(200000,)
(118850,)
